# Simulation: single ion Rabi with motional heating

In [96]:
from scipy import special
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
from tqdm.notebook import tqdm

## 1. Parameter definitions

### A. User-defined parameters

In [321]:
n_tot = 10               # total number of number state basis to represent the motional state: starts from zero
n_mean_init = 0         # initial mean number of the motional state
spin_down_init = 1      # initial population of spin down

f_Rabi = 50*10**3       # internal state Rabi frequency: units in Hz
f_secular = 1.6*10**6   # secular frequency: units in Hz
f_detuning = 0          # detuning: units in Hz
LD_param = 0.1          # Lamb-Dicke parameter: dimensionless
drift_phase = 0

heating_rate = 5000        # heating rate: units in phonons/sec
T_b = 300               # equilibrium temperature of the phonon heat bath: units in Kelvin

t_tot = 300*1e-6            # total time to simulate: units in sec
dt = 1e-9               # simulation time step: units in sec
t_scale = (10**6)*dt    # plot time scale: units in us

### B. Dependent parameters

In [322]:
# returns the mean number of the bath with respect to the motional energy
def return_bath_mean_n():
    hbar = 6.62607015*10**(-34)
    k_B = 1.380649*10**(-23)
    mean_n = 1/(np.exp(hbar*(2*np.pi*f_secular)/(k_B*T_b)) - 1)

    return mean_n

bath_mean_n = return_bath_mean_n()
print('bath mean number: ', round(bath_mean_n))

# returns the coupling efficiency to the bath 
def return_gamma():
    gamma = heating_rate/bath_mean_n

    return gamma

Gamma = return_gamma()
print('Gamma: ', round(Gamma, 6))

# returns the motional decay rate
def return_motional_decay_rate(n):
    decay_rate = Gamma*((2*bath_mean_n + 1)*n + bath_mean_n)/2
    
    return decay_rate

# returns the motional Rabi frequency for different number state transitions
def return_stationary_motional_f_Rabi(n_f, n_i):
    laguerre_factor_0 = complex(1)
    laguerre_factor_1 = complex(1)
    if n_f == n_i:
        laguerre_factor_0 = np.exp(-(LD_param**2)/2)*special.genlaguerre(n_i, 0)(LD_param**2)
        laguerre_factor_1 = laguerre_factor_0
    elif n_f > n_i:
        n_diff = n_f - n_i
        laguerre_factor = np.exp(-(LD_param**2)/2)*(special.factorial(n_i)/special.factorial(n_f))**(1/2)
        laguerre_factor_0 = laguerre_factor*((LD_param*1j)**n_diff)*special.genlaguerre(n_i, n_diff)(LD_param**2)
        laguerre_factor_1 = laguerre_factor*((-LD_param*1j)**n_diff)*special.genlaguerre(n_i, n_diff)(LD_param**2)
    elif n_f < n_i:
        n_diff = n_i - n_f
        laguerre_factor = np.exp(-(LD_param**2)/2)*(special.factorial(n_f)/special.factorial(n_i))**(1/2)
        laguerre_factor_0 = laguerre_factor*((LD_param*1j)**n_diff)*special.genlaguerre(n_f, n_diff)(LD_param**2)
        laguerre_factor_1 = laguerre_factor*((-LD_param*1j)**n_diff)*special.genlaguerre(n_f, n_diff)(LD_param**2)

    return [laguerre_factor_0*(2*np.pi*f_Rabi), laguerre_factor_1*(2*np.pi*f_Rabi)]

modified_f_Rabi2 = np.zeros((n_tot, n_tot, 2, 2), dtype=complex)
for m in range(n_tot):
    for n in range(n_tot):
        modified_f_Rabi2[m,n] = return_stationary_motional_f_Rabi(m, n)

modified_f_Rabi = list()
for m in range(n_tot):
    f_Rabi_row = list()
    for n in range(n_tot):
        f_Rabi_row.append(return_stationary_motional_f_Rabi(m, n))
    modified_f_Rabi.append(f_Rabi_row)
modified_f_Rabi = np.array(modified_f_Rabi)
    
def return_evolving_motional_f_Rabi(n_f, n_i, time):
    base_rabi = modified_f_Rabi[n_f][n_i]
    time_factor_0 = np.exp(1j*2*np.pi*((n_f - n_i)*f_secular + f_detuning)*time)
    time_factor_1 = np.exp(1j*2*np.pi*((n_f - n_i)*f_secular - f_detuning)*time)

    return [base_rabi[0]*time_factor_0, base_rabi[1]*time_factor_1]

# def return_evolving_motional_f_Rabi(n_f, n_i, time):
#     time_factor_0 = np.exp(1j*2*np.pi*((n_f - n_i)*f_secular - f_detuning)*time)
#     time_factor_1 = np.exp(- 1j*2*np.pi*((n_f - n_i)*f_secular + f_detuning)*time)

#     return [time_factor_0, time_factor_1]

def return_starionary_motional_f_Rabi(n_f, n_i):
    base_rabi = modified_f_Rabi[n_f][n_i]
    return base_rabi

bath mean number:  621797
Gamma:  0.008041


## 2. Simulation

In [323]:
# initialize the density matrix elements
sim_init_array = list()
sim_empty_array = list()

sim_spin = np.array([[complex(spin_down_init), complex(0)], [complex(0), complex(1 - spin_down_init)]])
sim_spin.reshape(2, 2)
sim_empty = np.array([[complex(0), complex(0)], [complex(0), complex(0)]])
sim_empty.reshape(2, 2)

for m in range(n_tot):
    sim_array_row = list()
    sim_empty_row = list()
    for n in range(n_tot):
        prob_n = 0
        if m == n:
            prob_n = (1/(n_mean_init + 1))*(n_mean_init/(n_mean_init + 1))**n
        sim_array_row.append(prob_n*sim_spin)
        sim_empty_row.append(sim_empty)
    sim_array_row = np.array(sim_array_row)
    sim_array_row.reshape(2, 2*n_tot)
    sim_init_array.append(sim_array_row)
    sim_empty_row = np.array(sim_empty_row)
    sim_empty_row.reshape(2, 2*n_tot)
    sim_empty_array.append(sim_empty_row)
sim_init_array = np.array(sim_init_array)
sim_init_array.reshape(2*n_tot, 2*n_tot)
sim_empty_array = np.array(sim_empty_array)
sim_empty_array.reshape(2*n_tot, 2*n_tot)

sim_init_array /= sim_init_array.sum()

print(sim_init_array[0][0])
print(sim_init_array[n_tot - 1][n_tot - 1])
print(sim_empty_array[0][0])

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [324]:
# sim_timeline = {'time_step' : list(), 'prob_up' : list()}

# # update density matrix elements
# sim_prev = cp.deepcopy(sim_init_array)
# sim_next = cp.deepcopy(sim_empty_array)
# I = np.array([[complex(1), complex(0)], [complex(0), complex(1)]])

# time = 0
# for it in tqdm(range(round(t_tot/dt))): 
#     for m in range(n_tot):
#         for n in range(n_tot):
#             sum_K = np.array([[complex(0), complex(0)], [complex(0), complex(0)]]) 

#             for r in range(n_tot):
#                 rabi = return_evolving_motional_f_Rabi(m, r, time + it*dt)
#                 K = -(1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]])   
#                 if r == m:
#                     K -= return_motional_decay_rate(r)*I
#                 sum_K += np.matmul(K, sim_prev[r][n])

#             for l in range(n_tot):
#                 rabi = return_evolving_motional_f_Rabi(l, n, time + it*dt)
#                 K_d = (1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]]) 
#                 if l == n:
#                     K_d -= return_motional_decay_rate(l)*I
#                 sum_K += np.matmul(sim_prev[m][l], K_d)
                
#             J = np.array([[complex(0), complex(0)], [complex(0), complex(0)]])
#             J_d = np.array([[complex(0), complex(0)], [complex(0), complex(0)]])
            
#             if (m < n_tot - 1) and (n < n_tot - 1):
#                 J = Gamma*(bath_mean_n + 1)*(((m + 1)*(n + 1))**(1/2))*sim_prev[m + 1][n + 1]
#             if (1 <= m) and (1 <= n):
#                 J_d = Gamma*bath_mean_n*((m*n)**(1/2))*sim_prev[m - 1][n - 1]

#             sim_next[m][n] = sim_prev[m][n] + (sum_K + J + J_d)*dt
    
#     traced = np.array([[complex(0), complex(0)], [complex(0), complex(0)]])
#     for diag_n in range(n_tot):
#         traced += sim_next[diag_n][diag_n]
#     prob_up = traced[1][1].real

#     sim_timeline['time_step'].append(it*t_scale)      # plot units in us
#     sim_timeline['prob_up'].append(prob_up)
    
#     sim_prev = cp.deepcopy(sim_next)
#     sim_next = cp.deepcopy(sim_empty_array)

# sim_final_array = cp.deepcopy(sim_prev)


## Matrix version

In [325]:
K_matrix = np.zeros_like(sim_empty_array, dtype=complex)
K_d_matrix = np.zeros_like(sim_empty_array, dtype=complex)
for m in range(n_tot):
    for n in range(n_tot):
        rabi = return_stationary_motional_f_Rabi(m, n)
        K_matrix[m,n] = -(1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]])    
        K_d_matrix[m,n] = (1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]]) 
        if m == n:
            K_matrix[m,n] -= return_motional_decay_rate(n)*np.eye(2, dtype=complex)
            K_d_matrix[m,n] -= return_motional_decay_rate(n)*np.eye(2, dtype=complex)

In [326]:
J_matrix = np.zeros((n_tot,n_tot), dtype=complex)
J_d_matrix = np.zeros((n_tot,n_tot), dtype=complex)
for m in range(n_tot):
    for n in range(n_tot):        
        if (m < n_tot - 1) and (n < n_tot - 1):
            J_matrix[m,n] = Gamma*(bath_mean_n + 1)*(((m + 1)*(n + 1))**(1/2))
        if (1 <= m) and (1 <= n):
            J_d_matrix[m,n] = Gamma*bath_mean_n*((m*n)**(1/2))            

In [327]:
# idx1, idx2 = 0, 0

In [328]:
# np.einsum('ijkl,ijlh->ijkh', sim_prev, K_matrix)[idx1][idx2]

In [329]:
# np.matmul(sim_prev[idx1][idx2], K_matrix[idx1][idx2])

In [330]:
# for idx1 in range(10):
#     for idx2 in range(10):
#         print(np.einsum('ijkl,ijlh->ijkh', sim_prev, K_matrix)[idx1][idx2] == np.matmul(sim_prev[idx1][idx2], K_matrix[idx1][idx2]))

In [331]:
# rabi_matrix1 = np.zeros((n_tot, n_tot, 2, 2), dtype=complex)
# rabi_matrix2 = np.zeros((n_tot, n_tot, 2, 2), dtype=complex)
# for m in range(n_tot):
#     for n in range(n_tot):
#         rabi = return_evolving_motional_f_Rabi(m, n, dt)
#         rabi_matrix1[m,n] = np.array([[1, rabi[0]], [rabi[1], 1]], dtype=complex)   
#         rabi = return_evolving_motional_f_Rabi(m, n, dt/2)
#         rabi_matrix2[m,n] = np.array([[1, rabi[0]], [rabi[1], 1]], dtype=complex)   

In [332]:
def get_evolved_K(K, t):
#     if idx == 1:
#         rabi_matrix = rabi_matrix1.copy()
#     if idx == 2:
#         rabi_matrix = rabi_matrix2.copy()
#     rabi_matrix = rabi_matrix2.copy()
#     K_matrix_t = np.einsum('ijkl,ijkl->ijkl', rabi_matrix, K_matrix)
#     K_d_matrix_t = np.einsum('ijkl,ijkl->ijkl', -rabi_matrix, K_d_matrix)
    
    K_matrix_t = np.zeros_like(sim_empty_array, dtype=complex)
    K_d_matrix_t = np.zeros_like(sim_empty_array, dtype=complex)
    for m in range(n_tot):
        for n in range(n_tot):
            rabi = return_evolving_motional_f_Rabi(m, n, t)
            K_matrix_t[m,n] = -(1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]])    
            K_d_matrix_t[m,n] = (1j/2)*np.array([[complex(0), rabi[0]], [rabi[1], complex(0)]]) 
            if m == n:
                K_matrix_t[m,n] -= return_motional_decay_rate(n)*np.eye(2, dtype=complex)
                K_d_matrix_t[m,n] -= return_motional_decay_rate(n)*np.eye(2, dtype=complex)
    return K_matrix_t, K_d_matrix_t

In [333]:
# def get_summed_term(rho_prev, transition = 'carrier'):
#     if transition == 'blue':
#         K_matrix_shift = np.pad(K_matrix,((0,0),(1,0),(0,0),(0,0)), mode='constant')[:, :-1]
#         rho_shift1 = np.pad(rho_prev,((1,0),(0,0),(0,0),(0,0)), mode='constant')[:-1, :]
#         K_d_matrix_shift = np.pad(K_d_matrix,((0,1),(0,0),(0,0),(0,0)), mode='constant')[1:, :]
#         rho_shift2 = np.pad(rho_prev,((0,0),(0,1),(0,0),(0,0)), mode='constant')[:, 1:]
#         K_term1 = np.einsum('ijkl,ijlh->ijkh', K_matrix_shift, rho_shift1)
#         K_term2 = np.einsum('ijkl,ijlh->ijkh', rho_shift2, K_d_matrix_shift)
#     elif transition == 'red':
#         K_matrix_shift = np.pad(K_matrix,((0,0),(0,1),(0,0),(0,0)), mode='constant')[:, 1:]
#         rho_shift1 = np.pad(rho_prev,((0,1),(0,0),(0,0),(0,0)), mode='constant')[1:, :]
#         K_d_matrix_shift = np.pad(K_d_matrix,((1,0),(0,0),(0,0),(0,0)), mode='constant')[:-1, :]
#         rho_shift2 = np.pad(rho_prev,((0,0),(1,0),(0,0),(0,0)), mode='constant')[:, :-1]
#         K_term1 = np.einsum('ijkl,ijlh->ijkh', K_matrix_shift, rho_shift1)
#         K_term2 = np.einsum('ijkl,ijlh->ijkh', rho_shift2, K_d_matrix_shift)
#     else:
#         K_term1 = np.einsum('ijkl,ijlh->ijkh', K_matrix, rho_prev)
#         K_term2 = np.einsum('ijkl,ijlh->ijkh', rho_prev, K_d_matrix)
        
#     rho_n_plus = np.pad(rho_prev,((0,1),(0,1),(0,0),(0,0)), mode='constant')[1:, 1:] # rho[m+1, n+1]
#     rho_n_minus = np.pad(rho_prev,((1,0),(1,0),(0,0),(0,0)), mode='constant')[:-1, :-1] # rho[m-1, n-1]
    
#     J_term1 = np.einsum('ij,ijkh->ijkh', J_matrix, rho_n_plus)
#     J_term2 = np.einsum('ij,ijkh->ijkh', J_d_matrix, rho_n_minus)
#     return K_term1 + K_term2 + J_term1 + J_term2

In [334]:
def get_summed_term(rho_prev, K_mat, K_d_mat):
    K_term1 = np.einsum('ijkl,ijlh->ijkh', K_mat, rho_prev)
    K_term2 = np.einsum('ijkl,ijlh->ijkh', rho_prev, K_d_mat)    
    
    rho_n_minus = np.pad(rho_prev,((0,1),(0,1),(0,0),(0,0)), mode='constant')[1:, 1:] # rho[m+1, n+1]
    rho_n_plus = np.pad(rho_prev,((1,0),(1,0),(0,0),(0,0)), mode='constant')[:-1, :-1] # rho[m-1, n-1]

    J_term1 = np.einsum('ij,ijkh->ijkh', J_matrix, rho_n_plus)
    J_term2 = np.einsum('ij,ijkh->ijkh', J_d_matrix, rho_n_minus)
    
#     J_term1 = J_coeff1 * rho_n_plus
#     J_term2= J_coeff2 * rho_n_minus
    
    return K_term1 + K_term2 + J_term1 + J_term2

In [335]:
transition = 'blue'

if transition == 'blue':
    f_detuning = f_secular
elif transition == 'red':
    f_detuning = -f_secular
else:
    f_detuning = 0

In [ ]:
sim_timeline = {'time_step' : list(), 'prob_up' : list(), 'rho_hist': []}

traced = sim_init_array.sum(axis=(0,1))
prob_up = traced[1][1].real
sim_timeline['time_step'].append(0)      # plot units in us
sim_timeline['prob_up'].append(prob_up)

# update density matrix elements
sim_prev = cp.deepcopy(sim_init_array)
sim_next = cp.deepcopy(sim_empty_array)
sim_timeline['rho_hist'].append(sim_prev.copy())

time = 0
for it in tqdm(range(round(t_tot/dt))): 
    
#     K_term1 = np.einsum('ijkl,ijlh->ijkh', K_matrix, sim_prev)
#     K_term2 = np.einsum('ijkl,ijlh->ijkh', sim_prev, K_d_matrix)
    
#     rho_n_plus = np.pad(sim_prev,((0,1),(0,1),(0,0),(0,0)), mode='constant')[1:, 1:] # rho[m+1, n+1]
#     rho_n_minus = np.pad(sim_prev,((1,0),(1,0),(0,0),(0,0)), mode='constant')[:-1, :-1] # rho[m-1, n-1]
    
#     J_term1 = np.einsum('ij,ijkh->ijkh', J_matrix, rho_n_plus)
#     J_term2 = np.einsum('ij,ijkh->ijkh', J_d_matrix, rho_n_minus)
    
#     summed = get_summed_term(sim_prev)
#     sim_next = sim_prev + summed*dt
    
    summed = get_summed_term(sim_prev, K_matrix, K_d_matrix)
    k1 = summed * dt
    K_matrix, K_d_matrix = get_evolved_K(K_matrix, it*dt + dt/2)
    summed = get_summed_term(sim_prev + k1/2, K_matrix, K_d_matrix)
    k2 = summed * dt
    summed = get_summed_term(sim_prev + k2/2, K_matrix, K_d_matrix)
    k3 = summed * dt
    K_matrix, K_d_matrix = get_evolved_K(K_matrix, it*dt + dt)
    summed = get_summed_term(sim_prev + k3, K_matrix, K_d_matrix)
    k4 = summed * dt
    
    sim_next = sim_prev + (k1 + 2*k2 + 2*k3 + k4)/6
#     sim_next /= sim_next.sum(axis=(0,1))[0,0] + sim_next.sum(axis=(0,1))[1,1]
    
    traced = sim_next.sum(axis=(0,1))
    prob_up = traced[1][1].real
    
    sim_timeline['time_step'].append((it+1)*t_scale)      # plot units in us
    sim_timeline['prob_up'].append(prob_up)
    sim_timeline['rho_hist'].append(sim_next.copy())
    
    sim_prev = cp.deepcopy(sim_next)
    sim_next = cp.deepcopy(sim_empty_array)

sim_final_array = cp.deepcopy(sim_prev)

  0%|          | 0/300000 [00:00<?, ?it/s]

In [ ]:
plt.rcParams['figure.figsize'] = (25,6)
plt.grid()
plt.xlim(0, sim_timeline['time_step'][-1])
# plt.ylim(-0.1, 1.1)
plt.plot(sim_timeline['time_step'], sim_timeline['prob_up'])

In [ ]:
rabi

In [ ]:
plt.plot(sim_timeline['rho_hist'][-1][:,:,0,0].flatten())

In [ ]:
plt.plot(sim_timeline['rho_hist'][-1][:,:,1,1].flatten())

In [ ]:
sim_timeline['rho_hist'][10].sum(axis=(0,1))

In [ ]:
sim_timeline['rho_hist'][10].sum(axis=(0,1))[0,0] + sim_timeline['rho_hist'][10].sum(axis=(0,1))[1,1]